In [2]:
import json
import csv

### Importing related matches

In [3]:
with open('/Users/anesterov/reps/wordsmatter/related_matches/rm.json','r') as jf:
    rms = json.load(jf)

In [7]:
# related matches from ODWN are in a separate file
with open('/Users/anesterov/reps/LODlit/bg/related_matches_odwn.json','r') as jf:
    rms_odwn = json.load(jf)

### Importing search results and generating a subset with related matches for every resource

In [61]:
def generate_rm_subset(resource_data:dict, rm_data:dict, resource:str, lang:str) -> dict:
    '''
    '''
    results = {}

    for query_term, value in resource_data.items():
        
        hits_by_query_term = []
    
        for rm_info in rm_data.values():
            
            if rm_info['lang'] == lang and query_term in rm_info['query_terms']:
                for hit in value:
                    
                    # checking resource
                    
                    if resource == 'wikidata':
                        
                        if hit.get('QID') == rm_info['related_matches']['wikidata'][0]:
                            hits_by_query_term.append(hit)
                            
                    if resource == 'aat':
                        
                        if hit.get('aat_uri') == rm_info['related_matches']['aat'][0]:
                            hits_by_query_term.append(hit)
                            
                    if resource == 'pwn':
                        
                        if hit.get('synset_id') in rm_info['related_matches']['pwn']:
                            hits_by_query_term.append(hit)
                            
                    # ODWN is handled separately
                            
        results[query_term] = hits_by_query_term
        
    return results

#### Wikidata EN

In [11]:
with open('/Users/anesterov/LODlit_local/wd/jan31/results_clean_en.json','r') as jf:
    wd_en = json.load(jf)

In [62]:
rm_wd_en = generate_rm_subset(wd_en,rms,"wikidata","en")

In [65]:
with open('/Users/anesterov/reps/LODlit/rm/rm_wd_en.json', 'w') as jf:
    json.dump(rm_wd_en, jf)

#### Wikidata NL

In [52]:
with open('/Users/anesterov/LODlit_local/wd/jan31/results_clean_nl.json','r') as jf:
    wd_nl = json.load(jf)

In [66]:
rm_wd_nl = generate_rm_subset(wd_nl,rms,"wikidata","nl")

In [67]:
with open('/Users/anesterov/reps/LODlit/rm/rm_wd_nl.json', 'w') as jf:
    json.dump(rm_wd_nl, jf)

#### AAT EN

In [55]:
with open('/Users/anesterov/reps/LODlit/AAT/aat_query_results_en.json','r') as jf:
    aat_en = json.load(jf)

In [68]:
rm_aat_en = generate_rm_subset(aat_en,rms,"aat","en")

In [69]:
with open('/Users/anesterov/reps/LODlit/rm/rm_aat_en.json', 'w') as jf:
    json.dump(rm_aat_en, jf)

#### AAT NL

In [70]:
with open('/Users/anesterov/reps/LODlit/AAT/aat_query_results_nl.json','r') as jf:
    aat_nl = json.load(jf)

In [71]:
rm_aat_nl = generate_rm_subset(aat_nl,rms,"aat","nl")

In [72]:
with open('/Users/anesterov/reps/LODlit/rm/rm_aat_nl.json', 'w') as jf:
    json.dump(rm_aat_nl, jf)

#### PWN

In [73]:
with open('/Users/anesterov/reps/LODlit/PWN/pwn31_query_results.json','r') as jf:
    pwn = json.load(jf)

In [74]:
rm_pwn = generate_rm_subset(pwn,rms,"pwn","en")

In [75]:
with open('/Users/anesterov/reps/LODlit/rm/rm_pwn.json', 'w') as jf:
    json.dump(rm_pwn, jf)

#### ODWN

In [76]:
with open('/Users/anesterov/reps/LODlit/ODWN/odwn_query_results.json','r') as jf:
    odwn = json.load(jf)

In [79]:
rm_odwn = {}

for query_term, value in odwn.items():
    
    hits_by_query_term = []
    
    for rm_info in rms_odwn.values():
        if query_term in rm_info['query_terms']:
            
            for hit in value:
                # all related matches in ODWN have LE_id
                if hit.get('le_id') in rm_info['odwn_le']:
                    hits_by_query_term.append(hit)
                    
    rm_odwn[query_term] = hits_by_query_term

In [81]:
with open('/Users/anesterov/reps/LODlit/rm/rm_odwn.json', 'w') as jf:
    json.dump(rm_odwn, jf)